In [22]:
import os
import json
import fiona
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

from shapely.geometry import shape
from shapely.geometry import Point

# Gee and EE 
import ee

# Clustering (best for testing** due to speed)
from sklearn.cluster import KMeans

# tif file creation
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

# Plotting and Vis 
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import re

In [23]:
# Initialise
ee.Authenticate()
ee.Initialize(project="jameswilliamchamberlain")

In [24]:
# basemap 
basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

# centeral point of Samarra 
df_sites = pd.DataFrame({
    "longitude": [43.823543],
    "latitude": [34.340989],
    "name": ["Samarra Archaeological City"],
    "category": ["Cultural"],
    "date inscribed": ["2007"],
    "region": ["Arab States"],
    "url": ["https://whc.unesco.org/en/list/276"],
    "iso": [["IQ"]]
})

# Chunks of Samarra Archaeological City 
with fiona.open("chunks_new.shp") as src:
    chunks = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [25]:
## TEMP REMOVE ADDITIONAL STUFF FOR QUICKER TESTING 
# tile_43641125_34108721_43915744_34336837 only 

chunks = chunks[chunks['file_name'] == 'tile_43641125_34108721_43915744_34336837']

In [26]:
m = geemap.Map()

if df_sites.empty:
    print("No sites found for the specified URL.")
else:
    m.add_points_from_xy(df_sites, x="longitude", y="latitude", layer_name="Sites")
    center_points = df_sites[['longitude', 'latitude']].mean().values
    m.setCenter(center_points[0], center_points[1], 10)

m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

# add chunks from aoi 
m.add_gdf(chunks, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})

m

Map(center=[34.340989, 43.823543], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [27]:
def create_subregions(chunks, sift_percentage_lon=0.5, sift_percentage_lat=0.415):
    """
        Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

        Assumes aoi contains all similar polygons, and are similar to a square.

        Parameters:
            aoi (GeoDataFrame): The area of interest containing geometries.
            sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
    """

    if chunks.empty:
        return {}
    
    first_polygon = chunks.geometry.iloc[0]

    # take top two points of the polygon and get the length between them
    top_points = first_polygon.exterior.coords[:2]
    length_lon = abs(top_points[0][0] - top_points[1][0])

    # calculate the shift amount
    shift_amount_lon = length_lon * sift_percentage_lon 
    shift_amount_lat = length_lon * sift_percentage_lat

    shift_directions = {
        "left": (-shift_amount_lon, 0),
        "right": (shift_amount_lon, 0),
        "up": (0, shift_amount_lat),
        "down": (0, -shift_amount_lat),
        "top_left": (-shift_amount_lon, shift_amount_lat),
        "top_right": (shift_amount_lon, shift_amount_lat),
        "bottom_left": (-shift_amount_lon, -shift_amount_lat),
        "bottom_right": (shift_amount_lon, -shift_amount_lat),
    }

    # shift_directions = {
    #     "left": (-shift_amount_lon, 0),
    #     "right": (shift_amount_lon, 0),
    #     "up": (0, shift_amount_lon),
    #     "down": (0, -shift_amount_lon),
    #     "top_left": (-shift_amount_lon, shift_amount_lon),
    #     "top_right": (shift_amount_lon, shift_amount_lon),
    #     "bottom_left": (-shift_amount_lon, -shift_amount_lon),
    #     "bottom_right": (shift_amount_lon, -shift_amount_lon),
    # }

    subregions = []
        
    # Create subregions by shifting the geometries in all directions
    for _, (dx, dy) in shift_directions.items():
        gdf_shifted = chunks.copy()
        gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
        subregions.append(gdf_shifted)

    # Combine all into one GeoDataFrame
    subregions = pd.concat(subregions, ignore_index=True)
    return gpd.GeoDataFrame(subregions, crs=chunks.crs)

def clip(chunks, aoi):
    """
        Clips the chunks or subregions to the area of interest (aoi).

        Parameters:
            chunks (GeoDataFrame):      The chunks or subregions to be clipped.
            aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
    """

    # clip to aoi 
    if chunks.empty or aoi.empty:
        return gpd.GeoDataFrame(columns=chunks.columns.tolist(), crs=chunks.crs)
    clipped = gpd.clip(chunks, aoi)
    clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

    return clipped.reset_index(drop=True)

subregions = create_subregions(chunks)
subregions = clip(subregions, aoi=chunks.dissolve())

# plot as one 
m.add_gdf(subregions, layer_name="Subregions", style={"color": "blue", "fillColor": "blue", "fillOpacity": 0.1})

# Reference Points

In [28]:
def collect_points_from_geemap(map_obj, label):
    """
    Collect all drawn point features from a geemap.Map that uses ee.Feature objects,
    and return them as a GeoDataFrame with a label.

    Parameters:
        map_obj (geemap.Map): The interactive map.
        label (str): Label to assign to all collected points.

    Returns:
        GeoDataFrame: With geometry and 'label' columns.
    """

    features = map_obj.draw_features

    if not features:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    points = []
    for f in features:
        try:
            geom = f.geometry()  # call the method
            if geom.getInfo()["type"] == "Point":
                coords = geom.coordinates().getInfo()  # [lon, lat]
                points.append(Point(coords))
        except Exception as e:
            print("Skipping feature due to error:", e)

    if not points:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    gdf = gpd.GeoDataFrame(geometry=points)
    gdf["label"] = label
    gdf.set_crs("EPSG:4326", inplace=True)
    return gdf

def gen_basemap(basemap_url=None, aoi=gpd.GeoDataFrame(), polygons=None):
    """
        Generates a basemap with the specified URL and adds polygons if provided.

        Parameters:
            basemap_url (str): The URL of the basemap to be used.
            polygons (GeoDataFrame, optional): Polygons to be added to the map. Defaults to None.
            aoi (GeoDataFrame): The area of interest to be displayed on the map. (can be a set of polygons or a single polygon)
        
        Returns:
            geemap.Map: A geemap map object with the specified basemap and polygons.
    """
    m = geemap.Map()

    if not basemap_url:
        basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
        
    m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

    # center on aoi
    if not aoi.empty:
        m.add_gdf(aoi, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
        center_points = aoi.geometry.unary_union.centroid.coords[0]
        m.setCenter(center_points[0], center_points[1], 10)
    
    if polygons is not None:
        m.add_gdf(polygons, layer_name="Polygons", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
    
    return m

In [29]:
# Example of 3 maps for urban, agricultrual, water and waste
urban_map = gen_basemap(basemap_url, aoi=chunks)
agricultural_map = gen_basemap(basemap_url, aoi=chunks)
water_map = gen_basemap(basemap_url, aoi=chunks)
wasteland_map = gen_basemap(basemap_url, aoi=chunks)
# wasteland_map = geemap.Map()

In [30]:
urban_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [31]:
agricultural_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [32]:
# water_map = water_waste_map
water_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [33]:
wasteland_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [34]:
# convert to labelled points in GeoDataFrame

water_points = collect_points_from_geemap(water_map, label="Water")
agricultural_points = collect_points_from_geemap(agricultural_map, label="Agricultural")
urban_points = collect_points_from_geemap(urban_map, label="Urban")
wasteland_map = collect_points_from_geemap(wasteland_map, label="Wasteland")


In [35]:
def save_points_to_csv(points, filename):
    points.to_csv(filename, index=False)
    print(f"Points saved to {filename}")

def load_points_from_csv(dir):
    try:
        points = gpd.read_file(dir)
        print(f"Points loaded from {dir}")
        return points
    except Exception as e:
        print(f"Error loading points from {dir}: {e}")
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

# save_points_to_csv(water_points, "test1_water_points.csv")
# save_points_to_csv(agricultural_points, "test1_agricultural_points.csv")
# save_points_to_csv(urban_points, "test1_urban_points.csv")
# save_points_to_csv(wasteland_map, "test1_wasteland_points.csv")

In [36]:
# merge into one GeoDataFrame
points_list = [water_points, agricultural_points, urban_points, wasteland_map]
labels = pd.concat(points_list, ignore_index=True)

# Clustering
Here we create a Sparse matrix

then with the sparse matrix compare and label based on that

In [37]:
pth = "data/"

# collect paths for all csv files in the folder
paths = []

for root, dirs, files in os.walk(pth):
    for file in files:
        if file.endswith(".csv"):
            paths.append(os.path.join(root, file))

print(f"Found {len(paths)} CSV files.")

# select 2024 
path = [p for p in paths if "2019" in p]
path


Found 6 CSV files.


['data/2019.csv']

In [38]:
def prep_data(dir):
    """Creates two dataframes from a CSV file, one ready for clustering with additional columns removed and the other with all columns left intact."""
    df1 = pd.read_csv(dir)
    df1 = df1.dropna()
    df2_clear = df1.copy()
    df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
    df2_clear = df2_clear.set_index("file_name")
    df1 = df1.set_index("file_name")
    df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

    return df1, df2_clear

df1, df2_clear = prep_data(path[0])

In [39]:

class tif_utils:
    """
        A set of utility functions for GeoTIFF files 
    """



    




# # predictions to tif
# output_dir = "/test1_1_predictions.tif"
# export_to_tif(predictions, bands=["numeric_label"], output_dir=output_dir, res=50, UTM_ESPG=32638, EPSG=4326)

In [ ]:
from collections import Counter

def kmeans_clustering(df, k=10):
    """sklearn kmeans"""

    kmeans = KMeans(n_clusters=k, random_state=42) # TODO: make random_state random
    kmeans.fit(df)
    
    return kmeans.labels_, df.index.tolist()

# def kmeans_clustering(df, **kwargs):
#     """sklearn kmeans with kwargs example for alternative functions in the future"""

#     import numpy as np
#     from sklearn.cluster import KMeans
    
#     k = kwargs.get('k', 10)
#     random_state = kwargs.get('random_state', np.random.randint(0, 1000))

#     kmeans = KMeans(n_clusters=k, random_state=random_state) 
#     kmeans.fit(df)
    
#     return kmeans.labels_, df.index.tolist()

class cluster:
    """
        Takes a set of polygons and a set of points with attached geometry 

        and runs clustering over all points in the polygons, and returns a set of clusters based on the points.
    """

    df_sparse = pd.DataFrame()

    def __init__(self, subregions, df_data, mapping, passes=6, aoi=None, index_column="file_name", points=gpd.GeoDataFrame()):
        """
            Initialises the cluster object with subregions, data, number of subregion passes, the area of interest (aoi), index column, and points.
        """

        # execution variables 
        self.passes = passes 

        # Data 
        self.data = df_data
        self.points = points
        self.mapping = mapping

        # Subregions 
        if aoi is None:
            aoi = subregions.dissolve()

        subregions_poly = create_subregions(chunks)
        subregions_poly = clip(subregions, aoi=aoi)
        self.subregions = subregions_poly

        self.index_column = index_column
        self.UTM_ESPG = 32638
        self.EPSG = 4326

        self.gdf_labels = None
        self.df_sparse = pd.DataFrame()
    
    def clip_dataframe(self, polygon, df):
        """
            Clips the DataFrame based on the .geo column and a given polygon.   
        
        Args:
            polygon (shapely.geometry.Polygon): The polygon to clip the DataFrame to.
            df (pd.DataFrame): The DataFrame containing the geometries to be clipped.
            
        Returns:
            pd.DataFrame: The clipped DataFrame.
        """

        # based on .geo column drop all rows that do not intersect with the polygon
        df['geometry'] = df['.geo'].apply(lambda x: shape(json.loads(x)))
        gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=self.UTM_ESPG)
        gdf = gdf[gdf.geometry.within(polygon)]
        gdf = gdf.drop(columns=['geometry'])

        # convert back to DataFrame
        df_clipped = pd.DataFrame(gdf)

        return df_clipped
    
    def clip_polygon(chunks, aoi):
        """
            Clips the chunks or subregions to the area of interest (aoi).

            Parameters:
                chunks (GeoDataFrame):      The chunks or subregions to be clipped.
                aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
        """

        # clip to aoi 
        if chunks.empty or aoi.empty:
            return gpd.GeoDataFrame(columns=chunks.columns.tolist(), crs=chunks.crs)
        clipped = gpd.clip(chunks, aoi)
        clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

        return clipped.reset_index(drop=True)
    
    def split_df(self, df):
        """
            Creates two dataframes from a CSV file, one ready for clustering with additional columns removed and the other with all columns left intact.
        """

        df1 = df.copy()

        df1 = df1.dropna()
        df2_clear = df1.copy()
        df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
        df2_clear = df2_clear.set_index("file_name")
        df1 = df1.set_index("file_name")
        df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

        return df1, df2_clear
    
    def create_subregions(chunks, sift_percentage=0.5):
        """
            TODO: Replace with updated version above
        """

        if chunks.empty:
            return {}
        
        first_polygon = chunks.geometry.iloc[0]

        # take top two points of the polygon and get the length between them
        top_points = first_polygon.exterior.coords[:2]
        length_lon = abs(top_points[0][0] - top_points[1][0])

        # calculate the shift amount
        shift_amount = length_lon * sift_percentage

        shift_directions = {
            "left": (-shift_amount, 0),
            "right": (shift_amount, 0),
            "up": (0, shift_amount),
            "down": (0, -shift_amount),
            "top_left": (-shift_amount, shift_amount),
            "top_right": (shift_amount, shift_amount),
            "bottom_left": (-shift_amount, -shift_amount),
            "bottom_right": (shift_amount, -shift_amount),
        }

        subregions = []

        # Create subregions by shifting the geometries in all directions
        for _, (dx, dy) in shift_directions.items():
            gdf_shifted = chunks.copy()
            gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
            subregions.append(gdf_shifted)

        # Combine all into one GeoDataFrame
        subregions = pd.concat(subregions, ignore_index=True)
        return gpd.GeoDataFrame(subregions, crs=chunks.crs)
    
    @staticmethod
    def convert_df_to_geodf(df, geo_col='.geo', crs="EPSG:32638"):
        """
        Converts a DataFrame with a '.geo' column (GeoJSON strings) to a GeoDataFrame.
        """
        df = df.copy()

        if df.empty:
            return gpd.GeoDataFrame(columns=df.columns.tolist(), crs=crs)
        if df is type(gpd.DataFrame):
            return df 

        # Only parse if the entry is a string
        def safe_parse(x):
            if isinstance(x, str):
                try:
                    return shape(json.loads(x))
                except Exception as e:
                    print(f"[WARNING] Bad geometry skipped: {x[:30]}... ({e})")
            return None

        df['geometry'] = df[geo_col].apply(safe_parse)
        df = df[df['geometry'].notnull()]  # drop invalid rows

        return gpd.GeoDataFrame(df, geometry='geometry', crs=crs)
    
    
    ##################################################################################################################################################################
    # ========================================================= BACKUP, RELOADING AND UPDATING DATA ================================================================ #
    ##################################################################################################################################################################    

    @staticmethod
    def save_processed_label_data(geodf, filename):
        """
            Saves the processed GeoDataFrame to a file.
        """

        if isinstance(geodf, gpd.GeoDataFrame):
            geodf.to_file(filename, driver='GeoJSON')
            print(f"GeoDataFrame saved to {filename}")
        elif isinstance(geodf, pd.DataFrame):
            geodf.to_csv(filename, index=True)
            print(f"DataFrame saved to {filename}")

    def reload(self, dir_data, dir_labels, dir_sparse, dir_points):
        """
            Reloads the data, labels and sparse DataFrame from the specified directories to resume processing.
        """

        # df_test1 = pd.read_csv("temp_test/test1_labels.geojson")
        # df_test1 = df_test1.drop(columns=["geometry"]) # does not work for some reason 
        # df_test1['geometry'] = df_test1['.geo'].apply(lambda x: shape(json.loads(x))) 
        # gpd_test1 = gpd.GeoDataFrame(df_test1, geometry='geometry', crs=c.UTM_ESPG)

        # data
        _temp_data = pd.read_csv(dir_data).drop(columns=["geometry"])
        _temp_data['geometry'] = _temp_data['.geo'].apply(lambda x: shape(json.loads(x)))
        _temp_data = gpd.GeoDataFrame(_temp_data, geometry='geometry', crs=self.UTM_ESPG)
        self.data = _temp_data

        # labels
        _temp_labels = pd.read_csv(dir_labels).drop(columns=["geometry"])
        _temp_labels['geometry'] = _temp_labels['.geo'].apply(lambda x: shape(json.loads(x)))
        _temp_labels = gpd.GeoDataFrame(_temp_labels, geometry='geometry', crs=self.UTM_ESPG)
        self.gdf_labels = _temp_labels

        # sparse 
        self.df_sparse = pd.read_csv(dir_sparse, index_col=0) # TODO: FIX BROKEN SAVE 

        # points
        with fiona.open(dir_points) as src:
            self.points = gpd.GeoDataFrame.from_features(src, crs=src.crs)

        # self.data = gpd.read_file(dir_data)
        # self.gdf_labels = gpd.read_file(dir_labels)
        # self.df_sparse = pd.read_csv(dir_sparse, index_col=0)
        # self.points = gpd.read_file(dir_points)

        print(f"Data reloaded from {dir_data}, {dir_labels} and {dir_sparse}.")

    ##################################################################################################################################################################
    # ================================================================== SAVE AND RESTORE ========================================================================== #
    ##################################################################################################################################################################

    

    ##################################################################################################################################################################
    # ================================================================== EXPORT FUNCTIONS ========================================================================== #
    ##################################################################################################################################################################

    # generate tif 
    @staticmethod
    def os_path_chcker(output_dir, postfix=".tif", NAME_CODE_LIM=8, FLAG_APPEND_POSTFIX=True):
        """
            Ensures that the output directory is valid, exists and can be written to.

            This is a simple function that does three things:
                1. Checks if the directory exists, if not creates it.
                2. Checks if the file name is valid (and not empty, else creates a unique filename).
                3. (Optional) Appends a postfix to the file name if it does not already end with it.

        Args:
            output_dir (str): The directory path to check.
            postfix (str, optional): The postfix to append to the file name if it does not already end with it. Default is ".tif".
            NAME_CODE_LIM (int, optional): The length of the hex code to generate for filename. Default is 8.
            FLAG_APPEND_POSTFIX (bool, optional): Whether to append the postfix if the file name does not end with it. Default is True.
        """


        # path existance check
        if not os.path.exists(os.path.dirname(output_dir)):
            os.makedirs(os.path.dirname(output_dir))
            print(f"Created directory: {os.path.dirname(output_dir)}")

        # file name existance check
        flag_filename = not os.path.basename(output_dir) or output_dir.endswith("/")
        while flag_filename:
            print("Generating new file name...")
            # assign unique name to the tif file
            new_file_name = f"{os.urandom(NAME_CODE_LIM).hex()}{postfix}"
            if not os.path.exists(output_dir + new_file_name):
                output_dir += new_file_name
                flag_filename = False
                print(f"New File Name: {output_dir}")

        # check labelled correctly 
        if FLAG_APPEND_POSTFIX and not output_dir.endswith(postfix):
            print(f"Output directory {output_dir} does not end with .tif, appending .tif")
            output_dir += {postfix}
            
        return output_dir

    def export_to_tif(self, df, bands, output_dir, res=50, UTM_ESPG=32638, EPSG=4326):
        """
            Exports a DataFrame generated by PlotToSat to a GeoTIFF file. 

            Note: df files MUST CONTAIN:
            - `.geo` column with GeoJSON geometries.
            - `file_name` column with unique identifiers for each row.
            - 1 Unique band labelled in `bands` list - this CAN BE LABELLED CLUSTERS. 

            Expected formats:
                Time-Series data should be in a DataFrame with 'month_band' columns (e.g., '0_B1', '1_B2', etc.).
                if a band is provideda e.g.m B1 but there are no pre-fix values this data will be assumed to be a single band AS-IS, and will be included in the output just as that band alone. 

            Args:
                df (pd.DataFrame): PlotToSat Style pandas DataFrame containing time-series data or single band data. Must have file_name and `.geo` geometry columns.
                bands (list): List of band names to include in the output. e.g., ['B1', 'B2', 'B3'] or ['SingleBand', etc.] can be mixed with single band data.
                output_dir (str): Output file directory for the GeoTIFF.
                res (int, optional): Resolution of the output raster in meters. Default is 50m.
                UTM_ESPG (int, optional): EPSG code for the UTM coordinate reference system. Default is 32638.
                


                # EPSG (int, optional): EPSG code for the coordinate reference system.
        """

        # Ensure output directory is valid, exists and can be written to.
        file = self.os_path_chcker(output_dir, postfix=".tif", NAME_CODE_LIM=8, FLAG_APPEND_POSTFIX=True)


        # Sort the Bands into single and time-series band data. 
        column_heads = df.columns.tolist()

        
        # all available bands in the data
        band_columns = [col for col in column_heads if any(col.endswith(band) for band in bands)]


        # Acceptable bands to process 
        acceptable_lst = []
        for col in bands:
            if any(col.startswith(f"{i}_") for i in range(12)):
                # must be non-prefix and only one band of that ts 
                if col not in acceptable_lst and any(coli for coli in band_columns if coli == col):
                    acceptable_lst.append(col)
                # acceptable_lst.append(col)
            else:
                # must exist in band_columns and take all 
                appended = False
                for i in range(12):
                    if any(coli for coli in band_columns if coli == f"{i}_{col}"):
                        if f"{i}_{col}" not in acceptable_lst:
                            # add only if not already in list     
                            acceptable_lst.append(f"{i}_{col}")
                            appended = True

                if appended == False:
                    # if not appended must be a unique column 
                    acceptable_lst.append(col)
        
        band_columns = acceptable_lst

    
        geometry = df[".geo"].apply(lambda x: shape(json.loads(x)))
        gdf = gpd.GeoDataFrame(df[band_columns].copy(), geometry=geometry, crs=f"EPSG:{EPSG}") # EPSG not UTM_ESPG else will raise an error

        gdf_utm = gdf.to_crs(epsg=UTM_ESPG)

        minx, miny, maxx, maxy = gdf_utm.total_bounds
        width = int((maxx - minx) / res)
        height = int((maxy - miny) / res)

        transform = from_origin(minx, maxy, res, res)

        if width <= 0 or height <= 0:
            raise ValueError(f"Invalid raster dimensions (width={width}, height={height}). Check CRS and resolution.")


        # Build Raster Stack
        rasters = []
        for band in band_columns:
            values = gdf_utm[band]
            shapes = ((geom, val) for geom, val in zip(gdf_utm.geometry, values))
            raster = rasterize(
                shapes=shapes,
                out_shape=(height, width),
                transform=transform,
                dtype="float32",
                fill=np.NaN,  # Fill with NaN values 
            )

            rasters.append(raster)

        raster_stack = raster_stack = np.stack(rasters, axis=0) if len(rasters) >= 2 else rasters


        # Save rasters to GeoTIFF 
        with rasterio.open(
            file,
            "w",
            driver="GTiff",
            height=height,
            width=width,
            count=len(band_columns),
            dtype="float32",
            crs=f"EPSG:{UTM_ESPG}",
            transform=transform,
            nodata=np.NaN,  # Set NoData value to NaN
        ) as dst:
            for i, band in enumerate(band_columns):
                dst.write(raster_stack[i], i + 1)
                dst.set_band_description(i + 1, band) # Keep band discription!!!


    def __convert_to_map__(self):
        """
        Converts the sparse DataFrame of multiple cluster label columns to a single-column label prediction
        based on majority voting from known labels.

        Args:
            df_sparse (pd.DataFrame): DataFrame with 'cluster_label_*' columns and index as file_name.
            df_data (pd.DataFrame): Original data containing '.geo' and any other metadata.
            gdf_labelled (GeoDataFrame): GeoDataFrame with true labels and index as file_name.

        Returns:
            pd.DataFrame: DataFrame indexed by file_name, with predicted label (or None if unassignable).
        """

        df_sparse = self.df_sparse.copy()
        df_data = self.data.copy()
        gdf_labelled = self.gdf_labels.copy()

        # Ensure labels column exists
        if 'class' not in gdf_labelled.columns:
            raise ValueError("gdf_labelled must contain a 'class' column with ground truth labels.")

        predictions = pd.Series(index=df_sparse.index, dtype=object)

        for col in df_sparse.columns:
            if not col.startswith("cluster_label_"):
                continue

            # Get cluster IDs and their associated true labels
            cluster_ids = df_sparse[col]
            known_labels = gdf_labelled['class']

            # Build mapping: cluster_id -> list of known labels
            cluster_to_labels = {}
            for file_name, cluster_id in cluster_ids.items():
                if file_name in known_labels and pd.notna(known_labels[file_name]):
                    cluster_to_labels.setdefault(cluster_id, []).append(known_labels[file_name])

            # Compute majority label for each cluster
            cluster_to_majority = {}
            for cluster_id, labels in cluster_to_labels.items():
                if not labels:
                    cluster_to_majority[cluster_id] = None
                else:
                    label_counts = Counter(labels)
                    most_common = label_counts.most_common()
                    top_label = most_common[0][0] if len(most_common) == 1 or most_common[0][1] != most_common[1][1] \
                        else np.random.choice([l for l, c in most_common if c == most_common[0][1]])
                    cluster_to_majority[cluster_id] = top_label

            # Assign predicted label per row
            for idx in df_sparse.index:
                cluster_id = df_sparse.at[idx, col]
                label = cluster_to_majority.get(cluster_id, None)
                if pd.isna(predictions.at[idx]) and label is not None:
                    predictions.at[idx] = label

        # Create output DataFrame
        result_df = pd.DataFrame({'predicted_label': predictions})

        # Join metadata like `.geo` if needed
        if '.geo' in df_data.columns:
            result_df = result_df.join(df_data['.geo'])

        return result_df


    def create_map(self, filename=""):
        predictions = self.__convert_to_map__()

        # tif creation
        predictions['numeric_label'] = predictions['predicted_label'].map(self.mapping)
        output_path = self.os_path_chcker(filename)
        self.export_to_tif(predictions, bands=['numeric_label'], output_dir=output_path)

        return predictions

    ##################################################################################################################################################################
    # =============================================================== PRE-PROCESSING =============================================================================== #
    ##################################################################################################################################################################

    def update_row_labels(self, gdf_labels):
        df_gdf = self.gdf_labels.copy()
        labels_gdf = gdf_labels.copy()

        df_gdf['label'] = None
        for _, label_row in labels_gdf.iterrows():
            print(f"progress: {_}/{len(labels_gdf)} labels processed.")
            label = label_row['label']
            
            # Check each point until assigned or dropped
            for idx, point in df_gdf.iterrows():
                if label_row.geometry.intersects(point.geometry):
                    df_gdf.at[idx, 'class'] = label
                    break

        self.gdf_labels = df_gdf

        return df_gdf


    def build_row_labels(self):
        """
        Builds a DataFrame with labelled polygons based on the points in the GeoDataFrame.

        Args:
            df (GeoDataFrame): The GeoDataFrame containing the points.
            labels (GeoDataFrame): The GeoDataFrame containing the polygons and their labels.

        Returns:
            pd.DataFrame: A DataFrame with the labels for each point.
        """


        # Prepare and Ensure in GeoDataFrame format
        df_gdf = self.data.copy()
        if df_gdf is type(pd.DataFrame):
            df_gdf = self.convert_df_to_geodf(df_gdf, geo_col='.geo', crs=f"EPSG:{self.UTM_ESPG}")
        labels_gdf = self.points.copy()

        # check for intersections and assign labels
        df_gdf['label'] = None
        for _, label_row in labels_gdf.iterrows():
            print(f"progress: {_}/{len(labels_gdf)} labels processed.")
            label = label_row['label']
            
            # Check each point until assigned or dropped
            for idx, point in df_gdf.iterrows():
                if label_row.geometry.intersects(point.geometry):
                    df_gdf.at[idx, 'class'] = label
                    break

        self.gdf_labels = df_gdf

        return df_gdf
    
    ##################################################################################################################################################################
    # ================================================================ Label Recomendations ======================================================================== #
    ##################################################################################################################################################################

    def create_recommendations(self, filename="/"):
        """
            Based on the Points Creates a map of recommendations for labelling the points in the subregions. 

            The higher the value the higher the recommendation to label the point. 
        """

        # resolution (m)
        res = 50

        # labels 
        labels = self.points.copy()
        labelled_data = self.gdf_labels.copy()
        data = self.data.copy()

        # # height map from truth labels 
        # max_height = 100.0
        # data['distance_value'] = 0.0 # old is 0.0

        # for _, label_geometry in labels.iterrows():
        #     print(f"progress: {_}/{len(labels)} labels processed.")
        #     label = label_geometry['label']
        #     if pd.isna(label):
        #         continue
            
        #     # Calculate distance map where lowest values are the points closest to the label geometry
        #     distances = data.geometry.distance(label_geometry.geometry)
        #     # data['distance_value'] += (max_height - distances) / max_height
        #     data['distance_value'] += distances / max_height

        # # normalise by the number of labels
        # if len(labels) > 0:
        #     data['distance_value'] /= len(labels)

        # # flip vlaues by max to create a recommendation map
        # data['distance_value'] = max_height - data['distance_value']


        max_height = 100.0
        data['distance_value'] = 0.0 

        for _, label_geometry in labels.iterrows():
            print(f"progress: {_}/{len(labels)} labels processed.")

            distances = data.geometry.distance(label_geometry.geometry) 

            # if distance is greater than 500m away ignore so set to 0 
            distances[distances > 500] = 0.0

            # normal around the point from 100 to 0.0 at edge
            data['distance_value'] += (max_height - distances) / max_height

            # take max value when compared between data['distance_value'] and the new value
            data['distance_value'] = data['distance_value'].combine(data['distance_value'], max)  
            
        try:
            self.export_to_tif(data, bands=['distance_value'], output_dir=self.os_path_chcker(filename, postfix=".tif", NAME_CODE_LIM=8, FLAG_APPEND_POSTFIX=True), res=50, UTM_ESPG=self.UTM_ESPG, EPSG=self.EPSG)
        except Exception as e:
            print(f"Error: {e}, \n Could not export recommendations to TIF, Returing DataFrame, so you can try again (This may be filename related!)")

        return data
        

    
    ##################################################################################################################################################################
    # ========================================================================== CLUSTERING ======================================================================== #
    ##################################################################################################################################################################
        
    def fit(self, cluster_fn, index_column='file_name'):
        """
            Fits the clustering model to the data.
            This method should be implemented in subclasses.
        """

        subregions_data = []

        for _, subregion in self.subregions.iterrows():
            # clip the data to the subregion
            # based on .geo column drop all rows that do not intersect with the polygon

            df_clipped = self.clip_dataframe(subregion.geometry, self.data)
            df_clipped = df_clipped.dropna()
            df_clipped = df_clipped.drop(columns=["system:index", ".geo"]) # unessusary columns for clustering
            # df_clipped = df_clipped.set_index("file_name")
            df_clipped = df_clipped.apply(pd.to_numeric, errors='coerce')

            if df_clipped.empty:
                print("No data points found in this subregion.")
                continue

            subregions_data.append(df_clipped)

        matrix = pd.DataFrame()
        index_number = 0

        for subregion_df in subregions_data:
            print(f"Clustering {len(subregion_df)} points from subregion.")

            for i in range(self.passes):
                # Perform clustering
                labels, indecies = kmeans_clustering(subregion_df, k=10)  # TODOP: replace with cluster_fn

                col_name = f'cluster_label_{index_number}'

                # Temporary DataFrame
                temp_df = pd.DataFrame({col_name: labels, 'file_name': indecies})

                # Group in case of duplicates
                temp_grouped = temp_df.groupby('file_name')[col_name].first()

                # Convert to DataFrame for merging
                temp_grouped = temp_grouped.to_frame()

                print(f"temp columns: {temp_grouped.columns}")

                # Merge with main matrix
                matrix = matrix.join(temp_grouped, how='outer') if not matrix.empty else temp_grouped

                index_number += 1
                print(len(matrix.columns), "columns in the matrix after clustering.")
                print(len(matrix), "rows in the matrix after clustering.")

        matrix = matrix.join(self.data[[".geo"]], how='outer')
        self.df_sparse = matrix

        return matrix
    

In [ ]:
mapping = {
            None: 0,
            np.NaN: 0,
            "NaN": 0,
            "None": 0,
            "Water": 1,
            "Agricultural": 2,
            "Urban": 3,
            "Wasteland": 4,
        }


# THIS WORKS 
c = cluster(chunks, df1, points=labels, mapping=mapping, passes=1)
c.reload("temp_test/test1_data.geojson", "temp_test/test1_labels.geojson", "temp_test/test1_sparse.csv", "temp_test/test1_points.geojson")
c.create_recommendations()
# matrix = c.fit(kmeans_clustering)
# c.create_map("/temptest2.tif")

# NOTE: test1_sparse.csv does NOT HAVE CORRECT DATA 



# c.build_row_labels() # TODO: add passes 
# self.data = gpd.read_file(dir_data)
# self.gdf_labels = gpd.read_file(dir_labels)
# self.df_sparse = pd.read_csv(dir_sparse, index_col=0)
# self.points = gpd.read_file(dir_points)

Data reloaded from temp_test/test1_data.geojson, temp_test/test1_labels.geojson and temp_test/test1_sparse.csv.
progress: 0/503 labels processed.
progress: 1/503 labels processed.
progress: 2/503 labels processed.
progress: 3/503 labels processed.
progress: 4/503 labels processed.
progress: 5/503 labels processed.
progress: 6/503 labels processed.
progress: 7/503 labels processed.
progress: 8/503 labels processed.
progress: 9/503 labels processed.
progress: 10/503 labels processed.
progress: 11/503 labels processed.
progress: 12/503 labels processed.
progress: 13/503 labels processed.
progress: 14/503 labels processed.
progress: 15/503 labels processed.
progress: 16/503 labels processed.
progress: 17/503 labels processed.
progress: 18/503 labels processed.
progress: 19/503 labels processed.
progress: 20/503 labels processed.
progress: 21/503 labels processed.
progress: 22/503 labels processed.
progress: 23/503 labels processed.
progress: 24/503 labels processed.
progress: 25/503 labels

In [ ]:
# OLD OLD OLD OLD OLD OLD OLD OLD 

def save_processed_label_data(geodf, filename):
    """Saves the processed GeoDataFrame to a file."""
    if isinstance(geodf, gpd.GeoDataFrame):
        geodf.to_file(filename, driver='GeoJSON')
        print(f"GeoDataFrame saved to {filename}")
    elif isinstance(geodf, pd.DataFrame):
        geodf.to_csv(filename, index=True)
        print(f"DataFrame saved to {filename}")

# save_processed_label_data(c.gdf_labels, "temp_test/test1_labels.geojson")
# save_processed_label_data(c.df_sparse, "temp_test/test1_sparse.csv")
# save_processed_label_data(c.data, "temp_test/test1_data.geojson")
# save_processed_label_data(c.points, "temp_test/test1_points.geojson")
# save_processed_label_data(c.subregions, "temp_test/test1_subregions.geojson")


# OLDER OLDER OLDER OLDER OLDER 
# # save c.df_sparse to a file #TODOFINDMEREF
# save_processed_label_data(c.df_sparse, "data/df_sparse_data.json")
# save_processed_label_data(c.gdf_labels, "data/labels.geojson")
###save_processed_label_data(c.data, "data/subregions.geojson") # may be useless 

In [ ]:
# wasteland_update_map = gen_basemap(basemap_url, aoi=chunks)
# wasteland_update_map

Map(center=[34.22282359393232, 43.778616017576226], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# wasteland_update_labels = collect_points_from_geemap(wasteland_update_map, label="Wasteland")
# c.update_row_labels(wasteland_update_labels)
# c.create_map("/temptest3.tif")

progress: 0/74 labels processed.
progress: 1/74 labels processed.
progress: 2/74 labels processed.
progress: 3/74 labels processed.
progress: 4/74 labels processed.
progress: 5/74 labels processed.
progress: 6/74 labels processed.
progress: 7/74 labels processed.
progress: 8/74 labels processed.
progress: 9/74 labels processed.
progress: 10/74 labels processed.
progress: 11/74 labels processed.
progress: 12/74 labels processed.
progress: 13/74 labels processed.
progress: 14/74 labels processed.
progress: 15/74 labels processed.
progress: 16/74 labels processed.
progress: 17/74 labels processed.
progress: 18/74 labels processed.
progress: 19/74 labels processed.
progress: 20/74 labels processed.
progress: 21/74 labels processed.
progress: 22/74 labels processed.
progress: 23/74 labels processed.
progress: 24/74 labels processed.
progress: 25/74 labels processed.
progress: 26/74 labels processed.
progress: 27/74 labels processed.
progress: 28/74 labels processed.
progress: 29/74 labels p

,predicted_label,.geo,numeric_label
0,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.99813739...",0
1,NaN,"{""type"":""Polygon"",""coordinates"":[[[44.10438435...",0
2,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.99813739...",0
3,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.99922097...",0
4,NaN,"{""type"":""Polygon"",""coordinates"":[[[44.00030452...",0
...,...,...,...
372308,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.91080588...",0
372309,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.91079693...",0
372310,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.91078360...",0
372311,NaN,"{""type"":""Polygon"",""coordinates"":[[[43.91077465...",0


In [ ]:
class ts_cluster():

    yearly_clusters = []

    def __init__(self, ts_point_labels, data_dir, mapping, passes=6):
        """
            Initializes the ts_cluster object with time-series point labels, data directory, mapping, and number of passes.
        """
        self.ts_point_labels = ts_point_labels          # expected labels in format year_start, year_end, label and point geometry 
        self.data_dir = data_dir                        # expected to have file_dir and year columns in pd.DataFrame format  
        self.mapping = mapping                          # mapping for labels to numeric values 
        self.passes = passes                            # int 
    

    def __create_shared_label_list__(self):
        """
            Creates a list of shared labels acrross ALL time-series points, such that it can act as a base for labels in the clustering process.
        """

        shared_labels = pd.DataFrame(columns=["year_start", "year_end", "label", "geometry"])
        non_shared_labels = pd.DataFrame(columns=["year_start", "year_end", "label", "geometry"])

        min_year = self.data_dir['year'].min()
        max_year = self.data_dir['year'].max()

        # if years between (incluside) are in ts_point_labels, then add to shared_labels else add to non_shared_labels
        for year in range(min_year, max_year + 1):
            if year in self.ts_point_labels['year_start'].values:
                shared_labels = shared_labels.append(self.ts_point_labels[self.ts_point_labels['year_start'] == year], ignore_index=True)

        # else 
        non_shared_labels = self.ts_point_labels[~self.ts_point_labels['year_start'].isin(shared_labels['year_start'])]

    

In [ ]:
x_1 = 2019
x_2 = 2024

for y in range(x_1, x_2 + 1):
    print(f"Processing year: {y}")

Processing year: 2019
Processing year: 2020
Processing year: 2021
Processing year: 2022
Processing year: 2023
Processing year: 2024
